In [1]:
import os
os.environ["BEEP_ENV"] = "dev"
os.environ["BEEP_PROCESSING_DIR"] ="data_transfer"
data_dir = "./data_transfer"

In [2]:
import pandas as pd
import numpy as np
import glob
import json
from datetime import datetime
from matplotlib import pyplot as plt
from monty.serialization import loadfn
from beep import structure, featurize

In [3]:
def get_time(d):
    ds = d.split(".")
    x = datetime.strptime(ds[0], '%Y-%m-%d %H:%M:%S')
    return x.timestamp() + float('.' + (ds[1]))

If trying to load in new timeseries.csv, make sure they are in data_transfer

In [4]:
data_dir = "./data_transfer"
file_list = glob.glob(os.path.join(data_dir, '*timeseries.csv'))
file_list = [f for f in file_list if "LFP" in f]
file_list

['./data_transfer\\SNL_18650_LFP_15C_0-100_0.5-1C_a_timeseries.csv',
 './data_transfer\\SNL_18650_LFP_15C_0-100_0.5-1C_b_timeseries.csv',
 './data_transfer\\SNL_18650_LFP_15C_0-100_0.5-2C_a_timeseries.csv',
 './data_transfer\\SNL_18650_LFP_15C_0-100_0.5-2C_b_timeseries.csv',
 './data_transfer\\SNL_18650_LFP_25C_0-100_0.5-0.5C_a_timeseries.csv',
 './data_transfer\\SNL_18650_LFP_25C_0-100_0.5-1C_a_timeseries.csv',
 './data_transfer\\SNL_18650_LFP_25C_0-100_0.5-1C_b_timeseries.csv',
 './data_transfer\\SNL_18650_LFP_25C_0-100_0.5-1C_c_timeseries.csv',
 './data_transfer\\SNL_18650_LFP_25C_0-100_0.5-1C_d_timeseries.csv',
 './data_transfer\\SNL_18650_LFP_25C_0-100_0.5-2C_a_timeseries.csv',
 './data_transfer\\SNL_18650_LFP_25C_0-100_0.5-2C_b_timeseries.csv',
 './data_transfer\\SNL_18650_LFP_25C_0-100_0.5-3C_a_timeseries.csv',
 './data_transfer\\SNL_18650_LFP_25C_0-100_0.5-3C_b_timeseries.csv',
 './data_transfer\\SNL_18650_LFP_25C_0-100_0.5-3C_c_timeseries.csv',
 './data_transfer\\SNL_18650_LFP

Cell below to generate csv files to run through structure and featurize from timeseries csv.

In [ ]:
file_list = glob.glob(os.path.join(data_dir, '*timeseries.csv'))
file_list = [f for f in file_list if "LFP" in f]
column_rename = {'Date_Time': 'datetime', 'Test_Time (s)':'test_time', 
                   'Cycle_Index':'cycle_index', 'Current (A)':'current',
                   'Voltage (V)':'voltage', 
                   'Charge_Capacity (Ah)':'charge_capacity',
                   'Discharge_Capacity (Ah)':'discharge_capacity',
                   'Charge_Energy (Wh)':'charge_energy', 
                   'Discharge_Energy (Wh)':'discharge_energy', 
                   'Cell_Temperature (C)':'temperature'}

new_dtypes = {'datetime': np.float32, 'test_time':np.float32, 
              'step_index':np.int16,
              'cycle_index': np.int32, 'current':np.float32,
              'voltage': np.float32, 
              'charge_capacity': np.float64,
              'discharge_capacity': np.float64,
              'charge_energy':np.float64, 
              'discharge_energy':np.float64, 
              'temperature':np.float32}
i = 0
for f in file_list:
    df = pd.read_csv(f)
    df = df.rename(columns=column_rename)
    df['step_index'] = 0
    df.datetime = df.datetime.apply(lambda x: get_time(x))
    df = df.astype(new_dtypes).drop('Environment_Temperature (C)', axis=1)
    if i < 10:
        j = "0" + str(i)
    else:
        j = str(i)
    new_name = "_".join(test.split("_")[3:8])+ "_CH"+ j
    df.to_csv("./data_transfer/%s.csv" %new_name, index=False)
    i += 1

Cell below to generate DeltaQ and Trajectory JSON files from newly created csv files

In [ ]:
file_list = glob.glob(os.path.join(data_dir, '*[0-9].csv'))
for f in file_list:
    print(f)
    raw_cycler_run = structure.RawCyclerRun.from_file(f)
    #raw_cycler_run.data = raw_cycler_run.data.dropna()
    processed_cycler_run = structure.ProcessedCyclerRun.from_raw_cycler_run(raw_cycler_run, [2.8, 3.5])
    
    #featurize 
    featurizer_DeltaQ = featurize.DeltaQFastCharge.from_run(input_filename='dummy.json', feature_dir='.', processed_cycler_run=processed_cycler_run)
    
    #export json file in correct format
    featurize_deltaQ_dict = {"@module": "beep.featurize", "@class": "DeltaQFastCharge"}
    featurize_deltaQ_dict["name"] = ('\\'.join(featurizer_DeltaQ.name.split("\\")[:-1]) +"\\" 
                                     + f.split("\\")[1][:-4] + "_features_DeltaQFastCharge.json")
    X_dict = {}
    for i in list(featurizer_DeltaQ.X.columns):
        X_dict[i] = [featurizer_DeltaQ.X.loc[0, i].astype(np.float64)]
    featurize_deltaQ_dict["X"] = X_dict
    featurize_deltaQ_dict["metadata"] = featurizer_DeltaQ.metadata
    featurize_deltaQ_dict["@version"] = "2021.1.11.15"
    json.dump(featurize_deltaQ_dict, open(featurize_deltaQ_dict['name'], 'w'))
    
    
    #export trajectory data
    featurize_Trajectory = featurize.TrajectoryFastCharge.from_run(input_filename=f.split("\\")[1][:-4] + '.json', 
                                                                   feature_dir='.', processed_cycler_run=processed_cycler_run)
    featurize_Trajectory_dict = {"@module": "beep.featurize", "@class": "TrajectoryFastCharge"}
    featurize_Trajectory_dict["name"] = ('\\'.join(featurize_Trajectory.name.split("\\")[:-1]) +"\\"+
                                         f.split("\\")[1][:-4] + "_features_TrajectoryFastCharge.json")
    X_dict = {}
    for i in list(featurize_Trajectory.X.columns):
        X_dict[i] = [featurize_Trajectory.X.loc[0, i].astype(np.float64)]
    featurize_Trajectory_dict["X"] = X_dict
    featurize_Trajectory_dict["metadata"] = featurize_Trajectory.metadata
    featurize_Trajectory_dict["@version"] = "2021.1.11.15"
    json.dump(featurize_Trajectory_dict, open(featurize_Trajectory_dict['name'], 'w'))
    

In [46]:
featurizer_DeltaQ = featurize.DeltaQFastCharge.from_run(input_filename='dummy.json', feature_dir='.', processed_cycler_run=processed_cycler_run)
featurizer_DeltaQ.X

c:\users\mattm\appdata\local\programs\python\python36\lib\site-packages\beep\featurize.py:1044: RuntimeWarning: All-NaN axis encountered
  X[17] = np.nanmin(IR_trend)


,discharge_capacity_cycle_2,max_discharge_capacity_difference,discharge_capacity_cycle_100,integrated_time_temperature_cycles_1:100,charge_time_cycles_1:5,abs_min_discharge_capacity_difference_cycles_2:100,abs_mean_discharge_capacity_difference_cycles_2:100,abs_variance_discharge_capacity_difference_cycles_2:100,abs_skew_discharge_capacity_difference_cycles_2:100,abs_kurtosis_discharge_capacity_difference_cycles_2:100,...,max_temperature_cycles_1:100,min_temperature_cycles_1:100,slope_discharge_capacity_cycle_number_2:100,intercept_discharge_capacity_cycle_number_2:100,slope_discharge_capacity_cycle_number_91:100,intercept_discharge_capacity_cycle_number_91:100,min_internal_resistance_cycles_2:100,internal_resistance_cycle_2,internal_resistance_difference_cycles_2:100,nominal_capacity_by_median
0,1.03,1.035,1.032,306697.588291,5760.0,-0.446806,-2.742859,-3.166992,0.584775,1.867028,...,18.459,15.16,-0.000595,1.072909,1.217792e-17,1.032,NaN,NaN,NaN,1.033


In [18]:
# featurize_deltaQ_dict = {"@module": "beep.featurize", "@class": "DeltaQFastCharge"}
# featurize_deltaQ_dict["name"] = ('\\'.join(featurizer_DeltaQ.name.split("\\")[:-1]) +"\\" 
#                                  + file_list[0].split("\\")[1][:-4] + "_features_DeltaQFastCharge.json")
# X_dict = {}
# for i in list(featurizer_DeltaQ.X.columns):
#     X_dict[i] = [featurizer_DeltaQ.X.loc[0, i].astype(np.float64)]
# featurize_deltaQ_dict["X"] = X_dict
# featurize_deltaQ_dict["metadata"] = featurizer_DeltaQ.metadata
# featurize_deltaQ_dict["@version"] = "2021.1.11.15"
# json.dump(featurize_deltaQ_dict, open(featurize_deltaQ_dict['name'], 'w'))
# featurize_deltaQ_dict

{'@module': 'beep.featurize',
 '@class': 'DeltaQFastCharge',
 'name': 'C:\\Users\\mattm\\Documents\\CalPoly\\DATA 451\\Toyota\\DeltaQFastCharge\\LFP_15C_0-100_0.5-1C_a_CH00_features_DeltaQFastCharge.json',
 'X': {'discharge_capacity_cycle_2': [1.03],
  'max_discharge_capacity_difference': [1.035],
  'discharge_capacity_cycle_100': [1.032],
  'integrated_time_temperature_cycles_1:100': [306697.58829142264],
  'charge_time_cycles_1:5': [5760.0],
  'abs_min_discharge_capacity_difference_cycles_2:100': [-0.44680629485828904],
  'abs_mean_discharge_capacity_difference_cycles_2:100': [-2.7428591200248187],
  'abs_variance_discharge_capacity_difference_cycles_2:100': [-3.16699189126899],
  'abs_skew_discharge_capacity_difference_cycles_2:100': [0.5847754492806106],
  'abs_kurtosis_discharge_capacity_difference_cycles_2:100': [1.867027981692003],
  'abs_first_discharge_capacity_difference_cycles_2:100': [-3.309824012313473],
  'max_temperature_cycles_1:100': [18.458999633789062],
  'min_temper

In [40]:
# featurize_Trajectory = featurize.TrajectoryFastCharge.from_run(input_filename=file_list[0].split("\\")[1][:-4] + '.json', 
#                                                                 feature_dir='.', processed_cycler_run=processed_cycler_run)
# featurize_Trajectory_dict = {"@module": "beep.featurize", "@class": "TrajectoryFastCharge"}
# featurize_Trajectory_dict["name"] = featurize_Trajectory.name
# # ('\\'.join(featurizer_Trajectory.name.split("\\")[:-1]) +"\\" 
# #                                  + file_list[0].split("\\")[1][:-4] + "_features_TrajectoryFastCharge.json")
# X_dict = {}
# for i in list(featurizer_Trajectory.X.columns):
#     X_dict[i] = [featurizer_Trajectory.X.loc[0, i].astype(np.float64)]
# featurize_Trajectory_dict["X"] = X_dict
# featurize_Trajectory_dict["metadata"] = featurizer_Trajectory.metadata
# featurize_Trajectory_dict["@version"] = "2021.1.11.15"
# json.dump(featurize_Trajectory_dict, open(featurize_Trajectory_dict['name'], 'w'))
# featurize_Trajectory_dict

{'@module': 'beep.featurize',
 '@class': 'TrajectoryFastCharge',
 'name': 'C:\\Users\\mattm\\Documents\\CalPoly\\DATA 451\\Toyota\\TrajectoryFastCharge\\LFP_15C_0-100.5-1C_a_features_TrajectoryFastCharge.json',
 'X': {'capacity_0.98': [506.0],
  'capacity_0.95': [506.0],
  'capacity_0.92': [506.0],
  'capacity_0.89': [4550.0],
  'capacity_0.86': [4550.0],
  'capacity_0.83': [4550.0],
  'capacity_0.8': [4550.0]},
 'metadata': {'barcode': None,
  'protocol': None,
  'channel_id': None,
  'parameters': {'thresh_max_cap': 0.98,
   'thresh_min_cap': 0.78,
   'interval_cap': 0.03}},
 '@version': '2021.1.11.15'}

In [25]:
variance_model = ["abs_variance_discharge_capacity_difference_cycles_2:100"]
variance_model_ind = [7]
discharge_model_ind = [5, 7, 8, 9, 0, 1]
full_model_ind = [5, 7, 13, 14, 0, 4, 3, 17, 19]

In [2]:
import pandas as pd
new_df = pd.read_csv("LFPBatteries.csv")

In [3]:
new_df

,abs_first_discharge_capacity_difference_cycles_2:100,abs_kurtosis_discharge_capacity_difference_cycles_2:100,abs_mean_discharge_capacity_difference_cycles_2:100,abs_min_discharge_capacity_difference_cycles_2:100,abs_skew_discharge_capacity_difference_cycles_2:100,abs_variance_discharge_capacity_difference_cycles_2:100,charge_time_cycles_1:5,discharge_capacity_cycle_100,discharge_capacity_cycle_2,integrated_time_temperature_cycles_1:100,...,nominal_capacity_by_median,slope_discharge_capacity_cycle_number_2:100,slope_discharge_capacity_cycle_number_91:100,capacity_0.8,capacity_0.83,capacity_0.86,capacity_0.89,capacity_0.92,capacity_0.95,capacity_0.98
0,-3.309824,1.867028,-2.742859,-0.446806,0.584775,-3.166992,5760.000000,1.032,1.030,306697.588291,...,1.0330,-0.000595,1.217792e-17,4550.0,4550.0,4550.0,4550.0,506.0,506.0,506.0
1,-3.019408,2.349816,-2.648706,-0.208776,0.993881,-3.056759,5785.600098,1.042,1.040,296427.402899,...,1.0440,-0.000603,-5.691578e-18,3552.0,3552.0,3552.0,3552.0,3552.0,506.0,506.0
2,-2.312774,1.694375,-1.917594,-0.195631,0.655882,-2.478396,5811.200195,1.030,1.030,262556.750559,...,1.0310,-0.000596,1.727324e-17,3760.0,3760.0,3760.0,3760.0,506.0,506.0,506.0
3,-2.677882,1.495850,-2.076420,-0.631730,0.057256,-2.981589,5785.600098,1.025,1.029,268508.727464,...,1.0280,-0.000626,1.943125e-18,3753.0,3753.0,3753.0,3753.0,506.0,506.0,506.0
4,-2.005443,0.393445,-1.682799,-0.189399,-0.220132,-1.291194,5785.600098,1.059,1.067,614925.496456,...,1.0670,-0.000724,-9.696970e-05,3049.0,3049.0,3049.0,3049.0,207.0,207.0,207.0
5,-2.380008,1.233238,-1.920131,-0.366808,0.547162,-2.368781,5785.600098,1.029,1.032,460906.194088,...,1.0320,-0.000626,-1.641712e-17,3544.0,3544.0,3544.0,3544.0,506.0,506.0,506.0
6,-2.515111,1.724767,-2.258376,-0.141387,0.812973,-2.289163,5785.600098,1.035,1.035,460296.723763,...,1.0380,-0.000614,-1.454545e-04,3547.0,3547.0,3547.0,3547.0,506.0,506.0,506.0
7,-0.943098,0.797936,-2.145172,-0.455522,-0.123878,-2.469690,5785.600098,1.048,1.050,457406.060653,...,1.0510,-0.000622,-1.030303e-04,3037.0,3037.0,3037.0,3037.0,3037.0,506.0,506.0
8,-0.897819,0.712504,-2.048117,-0.525766,-0.122899,-2.473751,5785.600098,1.047,1.049,467247.932532,...,1.0490,-0.000622,-1.381992e-17,3037.0,3037.0,3037.0,3037.0,3037.0,506.0,471.0
9,-2.451741,1.735859,-2.396135,-0.489405,0.711692,-3.078727,5811.200195,1.030,1.036,394481.761096,...,1.0330,-0.000623,-9.696970e-05,3543.0,3543.0,3543.0,3543.0,506.0,506.0,479.0
